
#### Author: Christopher Agard

In [11]:
import pandas as pd
import numpy as np
import os,glob,PyPDF2,camelot,ghostscript

pd.options.display.max_columns = 50

RuntimeError: Can not find Ghostscript DLL in registry

# Setting source and output folders

In [2]:
tu={'worksource': "C:/Users/tuh27554/Documents/Grading/2019/FALL EX1/Input Data",
    'workoutput': "C:/Users/tuh27554/Documents/Grading/2019/FALL EX1/Output Data"}
gandolf={'worksource': "C:/Users/craga/Google Drive/TU 2019/Fall/Exam 1/Input data",
    'workoutput': "C:/Users/craga/Google Drive/TU 2019/Fall/Exam 1/Output Data"}
source = tu

# Getting and Cleaning Exam Data

In [3]:
os.chdir(source['worksource'])
files = glob.glob('*.pdf')
files

['allTables.pdf',
 'Exam 1 Ver 1stats.pdf',
 'marcexams_2019_2019-A0153.pdf',
 'marcexams_2019_2019-A0154.pdf']

Create pdfFile Reader object

In [4]:
pdf1=PyPDF2.PdfFileReader(open(files[1],"rb"))

Get pages with tables (pages 5 and 6 in the pdf)

In [5]:
p5 = pdf1.getPage(4)
p6 = pdf1.getPage(5)

Create a writer object and add pages to it

In [ ]:
writer = PyPDF2.PdfFileWriter()
writer.addPage(p5)
writer.addPage(p6)

Write writer to a pdf file

In [ ]:
NewPDFfilename = "allTables.pdf" #filename of your PDF/directory where you want your new PDF to be
with open(NewPDFfilename, "wb") as outputStream: #create new PDF
    writer.write(outputStream) #write pages to new PDF

In [10]:
camelot.read_pdf('allTables.pdf')

RuntimeError: Please make sure that Ghostscript is installed

In [ ]:
# import tabula

# Read pdf into DataFrame
df = tabula.read_pdf("allTables.pdf", pages='all')

# # Read remote pdf into DataFrame
# df2 = tabula.read_pdf("https://github.com/tabulapdf/tabula-java/raw/master/src/test/resources/technology/tabula/arabic.pdf")

# # convert PDF into CSV
# tabula.convert_into("test.pdf", "output.csv", output_format="csv", pages='all')

# # convert all PDFs in a directory
# tabula.convert_into_by_batch("input_directory", output_format='csv', pages='all)

In [ ]:
tabula.read_pdf('allTables.pdf')

Now we change the dir to the outputfolder and save the cleaned, merged file.

In [ ]:
os.chdir(source['workoutput'])
df.to_csv("Fall19Exam 1_cleaned.csv")

# Analyzing Exam Data

In [ ]:
df.head()

# Most Frequently Wrong Answers

Here we identify the columns containing information about responses.

In [ ]:
itemcols = df.columns[df.columns.str.contains('item')]
itemcols

In [ ]:
df.soarType.value_counts()

def countwrong (df,col,ver,truthcol = 'soarType',truthind = 'key',compind = ['mine'],vercol = 'version'):
    truth = df.loc[(df[truthcol]==truthind)&(df[vercol] == ver),col]
    comp = df.loc[(df[truthcol].isin(compind))&(df[vercol] == ver),col]
    res = pd.DataFrame(data ={'item':[col],'number':[(comp != truth[0]).sum()]})
    return(res)


def countwrong_allcol(df,ver,itemcols,sortfields = ['number','item'],
                      truthcol = 'soarType',truthind = 'key',compind = 'mine',vercol = 'version'):
    tmp = pd.DataFrame()
    for col in itemcols:
        tmp = tmp.append(countwrong(df= df, col = col, ver = ver))
    tmp = tmp.sort_values(sortfields,ascending=False).reset_index(drop = True)
    return tmp

In [ ]:
help(countwrong)

In [ ]:
foo = pd.DataFrame(data={'col1':[1,2,3,4,5,1],'col2':[0,1,1,1,1,1]})
bar = foo.loc[foo.col2==0,'col1']
stuff = foo.loc[foo.col2==1,'col1']
bar[0]!=stuff

In [ ]:
countwrong(df,'item_20',1)

In [ ]:
v1wrong = countwrong_allcol(df,1,itemcols = df.columns[df.columns.str.contains('item')])
v2wrong = countwrong_allcol(df,2,itemcols = df.columns[df.columns.str.contains('item')])

Now we apply this function to create a ranked list of items by order of the number of my students getting them wrong.

In [ ]:
wrongcount_v1 = pd.DataFrame()
for col in itemcols:
    wrongcount_v1 = wrongcount_v1.append(countwrong(df= df, col = col, ver = 1,compind=['mine','other']))
wrongcount_v1 = wrongcount_v1.sort_values(['number','item'],ascending=False).reset_index(drop = True)
wrongcount_v1

In [ ]:
wrongcount_v2 = pd.DataFrame()
for col in itemcols:
    wrongcount_v2 = wrongcount_v2.append(countwrong(df= df, col = col, ver = 2,compind=['other','mine']))
wrongcount_v2 = wrongcount_v2.sort_values(['number','item'],ascending=False).reset_index(drop = True)
wrongcount_v2

## Sec 73

## Sec 74

## Sec 75

# Printing the Keys

In [ ]:
v1keys = df.loc[(df.tuid=='NNNNNNNNN')&(df.version==1),itemcols]
v1keys

In [ ]:
v2keys = df.loc[(df.tuid=='NNNNNNNNN')&(df.version==2),itemcols]
v2keys

In [ ]:
# v1keys = v1keys.T.rename(columns={0:'version1'})
# v2keys = v2keys.T.rename(columns={0:'version2'})
pd.DataFrame(data = {'version1':v1keys,'version2':v2keys},index = itemcols)

In [ ]:
letters2num_dict = {1:'D',2:'C',3:'B',4:'A'}
exkeys.version